## Install and Import Libraries

In [ ]:
# Install required packages

#%pip install ultralytics
#%pip install opencv-python
#%pip install cvzone
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [ ]:
# Import Essential Libraries
from ultralytics import YOLO
import cv2
import cvzone
import math
import torch


## Data Preparation

In [ ]:
# Check if GPU is available
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

# Load model and class names
model = YOLO("yolo-weights/yolo11l.pt")


In [ ]:
# Class names for COCO dataset
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", 
              "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", 
              "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", 
              "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", 
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", 
              "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]


## Training and Saving Model

In [ ]:
# Train Model
train_results = model.train(
    data="data/coco.yaml", 
    epochs=10, 
    imgsz=640,
)

# Evaluate model performance on the validation set
#metrics = model.val()

# Perform object detection on an image
#results = model("data/videos/test.png")
#results[0].show()


In [ ]:
# Save the model
#model.save("trained-models/yolo11l-trained.pt")


## Testing Model

In [ ]:
# Load camera and test the model

# Load camera
#cap = cv2.VideoCapture(0)
#cap.set(3, 640)
#cap.set(4, 360)

# Load video
cap = cv2.VideoCapture("data/videos/highway_test_video.mp4")

# Initiate and test the model
while True:
    success, img = cap.read()
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:

            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence and Class name box
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])

            objclass = classNames[cls]
            if objclass == "person" or objclass == "car" or objclass == "truck" or objclass == "bus" or objclass == "bicycle" or objclass == "motorbike" or objclass == "train":
                cvzone.cornerRect(img, (x1, y1, w, h), l = 8, t = 2)
                cvzone.putTextRect(img, f"{objclass} - {conf}", (max(0, x1), max(40, y1)), scale=0.6, thickness=1, offset=3)

    cv2.imshow("Image", img)
    cv2.waitKey(1)
